# Elastic Modulus Comparison

This notebook compares three different methods for calculating elastic modulus of snow layers:
1. **Bergfeld et al. (2023)** - Power-law based on density
2. **Köchle and Schneebeli (2014)** - Exponential relationships from μ-CT and FE simulations  
3. **Wautier et al. (2015)** - Power-law with ice modulus parameter

All three methods require both **density** and **grain form** as inputs.

This notebook shows a comparison of methods using density measurements that are directly input by the user, not calculated. 


In [1]:
# Setup and Imports
%load_ext autoreload
%autoreload 2

import sys
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from uncertainties import ufloat
from matplotlib.lines import Line2D
from matplotlib.patches import Patch

# Add the src directory to import snowpyt_mechparams
sys.path.append('../src')
from snowpilot_utils import convert_grain_form, parse_sample_pits
from snowpyt_mechparams import density, elastic_modulus


In [2]:
# Parse all snowpit files from the data folder
all_pits = parse_sample_pits('data')


Successfully parsed 50278 files
Failed to parse 0 files


In [3]:
# Collect layer information with direct density measurements
layer_info = []

for pit in all_pits:
    for layer in pit.snow_profile.layers:
        grain_form = layer.grain_form_primary.basic_grain_class_code if layer.grain_form_primary else None
        layer_dict = {
            'pit_id': pit.core_info.pit_id,
            'hand_hardness': layer.hardness,
            'depth_top': layer.depth_top[0] if layer.depth_top else None,  
            'thickness': layer.thickness[0] if layer.thickness else None,
            'grain_form': grain_form
        }
        
        # Get direct density measurement if available by matching depth_top and thickness
        layer_dict['density_direct'] = None
        for density_obs in pit.snow_profile.density_profile:
            if density_obs.depth_top == layer.depth_top and density_obs.thickness == layer.thickness:
                layer_dict['density_direct'] = density_obs.density
                break

        layer_info.append(layer_dict)

# Create dataframe
layer_df = pd.DataFrame(layer_info)


# Calculate Elastic Modulus using Three Methods

In [4]:
# Function to calculate elastic modulus for a single row using multiple methods
def calculate_emod_row(row):
    """Calculate elastic modulus for a single row using multiple methods."""
    result = {
        'e_mod_bergfeld': np.nan,
        'e_mod_kochle': np.nan,
        'e_mod_wautier': np.nan
    }
    
    # Skip if no grain form or density data
    # Handle both scalar and array values for density_direct
    density_val = row['density_direct']
    if pd.isna(row['grain_form']) or density_val is None:
        return pd.Series(result)
    
    # Check if density is an array/list and extract scalar if needed
    if isinstance(density_val, (list, np.ndarray)):
        if len(density_val) == 0 or pd.isna(density_val[0]):
            return pd.Series(result)
        density_val = density_val[0]
    elif pd.isna(density_val):
        return pd.Series(result)
    
    try:
        # Convert density to ufloat (no uncertainty available for direct measurements)
        density_ufloat = ufloat(density_val, 0)
        grain_form = row['grain_form']
        
        # Calculate elastic modulus using Bergfeld method
        try:
            E_mod_bergfeld = elastic_modulus.calculate_elastic_modulus(
                method='bergfeld',
                density=density_ufloat,
                grain_form=grain_form
            )
            result['e_mod_bergfeld'] = E_mod_bergfeld.nominal_value if hasattr(E_mod_bergfeld, 'nominal_value') else E_mod_bergfeld
        except Exception:
            pass
        
        # Calculate elastic modulus using Kochle method
        try:
            E_mod_kochle = elastic_modulus.calculate_elastic_modulus(
                method='kochle',
                density=density_ufloat,
                grain_form=grain_form
            )
            result['e_mod_kochle'] = E_mod_kochle.nominal_value if hasattr(E_mod_kochle, 'nominal_value') else E_mod_kochle
        except Exception:
            pass
        
        # Calculate elastic modulus using Wautier method
        try:
            E_mod_wautier = elastic_modulus.calculate_elastic_modulus(
                method='wautier',
                density=density_ufloat,
                grain_form=grain_form
            )
            result['e_mod_wautier'] = E_mod_wautier.nominal_value if hasattr(E_mod_wautier, 'nominal_value') else E_mod_wautier
        except Exception:
            pass
        
    except Exception:
        pass
    
    return pd.Series(result)

# Apply the function to all rows at once
layer_df[['e_mod_bergfeld', 'e_mod_kochle', 'e_mod_wautier']] = layer_df.apply(calculate_emod_row, axis=1)


/Users/marykateconnelly/Desktop/F25/SnowPyt-MechParams/venv/lib/python3.13/site-packages/uncertainties/core.py:1024: UserWarning: Using UFloat objects with std_dev==0 may give unexpected results.
  warn("Using UFloat objects with std_dev==0 may give unexpected results.")


# Analysis: Success Rates and Grain Form Compatibility


In [5]:
# Calculate success rates
total_layers = len(layer_df)
layers_with_data = layer_df[
    layer_df['density_direct'].notna() & 
    layer_df['grain_form'].notna()
].shape[0]

bergfeld_success = layer_df['e_mod_bergfeld'].notna().sum()
kochle_success = layer_df['e_mod_kochle'].notna().sum()
wautier_success = layer_df['e_mod_wautier'].notna().sum()

print("="*70)
print("SUCCESS RATE ANALYSIS")
print("="*70)
print(f"\nTotal layers: {total_layers:,}")
print(f"Layers with both density and grain form: {layers_with_data:,}")
print(f"\n{'Method':<15} {'Successful':<12} {'% of Total':<12} {'% of Available':<15}")
print("-"*70)
print(f"{'Bergfeld':<15} {bergfeld_success:<12,} {100*bergfeld_success/total_layers:>10.2f}% {100*bergfeld_success/layers_with_data:>13.2f}%")
print(f"{'Köchle':<15} {kochle_success:<12,} {100*kochle_success/total_layers:>10.2f}% {100*kochle_success/layers_with_data:>13.2f}%")
print(f"{'Wautier':<15} {wautier_success:<12,} {100*wautier_success/total_layers:>10.2f}% {100*wautier_success/layers_with_data:>13.2f}%")

# Overlaps
all_three = layer_df[
    layer_df['e_mod_bergfeld'].notna() & 
    layer_df['e_mod_kochle'].notna() & 
    layer_df['e_mod_wautier'].notna()
].shape[0]

any_two = layer_df[
    (layer_df['e_mod_bergfeld'].notna() & layer_df['e_mod_kochle'].notna()) |
    (layer_df['e_mod_bergfeld'].notna() & layer_df['e_mod_wautier'].notna()) |
    (layer_df['e_mod_kochle'].notna() & layer_df['e_mod_wautier'].notna())
].shape[0]

any_one = layer_df[
    layer_df['e_mod_bergfeld'].notna() | 
    layer_df['e_mod_kochle'].notna() | 
    layer_df['e_mod_wautier'].notna()
].shape[0]

print(f"\n{'Overlap Analysis':<40} {'Count':<12} {'% of Total'}")
print("-"*70)
print(f"{'At least one method succeeded':<40} {any_one:<12,} {100*any_one/total_layers:>10.2f}%")
print(f"{'At least two methods succeeded':<40} {any_two:<12,} {100*any_two/total_layers:>10.2f}%")
print(f"{'All three methods succeeded':<40} {all_three:<12,} {100*all_three/total_layers:>10.2f}%")


SUCCESS RATE ANALYSIS

Total layers: 371,429
Layers with both density and grain form: 9,880

Method          Successful   % of Total   % of Available 
----------------------------------------------------------------------
Bergfeld        3,132              0.84%         31.70%
Köchle          3,764              1.01%         38.10%
Wautier         8,354              2.25%         84.55%

Overlap Analysis                         Count        % of Total
----------------------------------------------------------------------
At least one method succeeded            8,611              2.32%
At least two methods succeeded           4,897              1.32%
All three methods succeeded              1,742              0.47%
